In [ ]:
# LSTM for Stock Market Prediction with Technical Indicators (no scaling)

import math
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

np.random.seed(42)
tf.random.set_seed(42)

# ----------------- Config -----------------
TICKERS = ["AAPL", "MSFT", "GOOG"]   # Run for multiple tickers
START = "2015-01-01"
END = "2025-01-01"
SEQ_LEN = 21
EPOCHS = 30
BATCH_SIZE = 32
# ------------------------------------------

# ----- Technical Indicator Functions -----
def add_indicators(df):
    df['MA10'] = df['Close'].rolling(10).mean()
    df['MA50'] = df['Close'].rolling(50).mean()

    # RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD (12-26) + Signal (9)
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Bollinger Bands (20-day)
    sma20 = df['Close'].rolling(20).mean()
    std20 = df['Close'].rolling(20).std()
    df['BB_upper'] = sma20 + (2 * std20)
    df['BB_lower'] = sma20 - (2 * std20)

    return df.dropna()


# ----- Sequence creation -----
def create_sequences(arr, seq_len, target_col_idx=3):  # Close = col 3
    X, y = [], []
    for i in range(len(arr) - seq_len):
        X.append(arr[i:i+seq_len])
        y.append(arr[i+seq_len, target_col_idx])
    return np.array(X), np.array(y)


# ----- Model -----
def build_lstm(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model


# ----- Run for each ticker -----
for ticker in TICKERS:
    print(f"\n===== Running LSTM + Technical Indicators for {ticker} =====")

    # 1) Download data
    df = yf.download(ticker, start=START, end=END, progress=False)
    df = df[['Open', 'High', 'Low', 'Close']].round(2).dropna()

    # 2) Add indicators
    df = add_indicators(df)

    # 3) Sequence creation
    values = df.values
    X, y = create_sequences(values, SEQ_LEN, 3)

    # Train-test split
    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    # 4) Model
    model = build_lstm((SEQ_LEN, X.shape[2]))
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

    # 5) Evaluate
    pred = model.predict(X_test).ravel()
    rmse = math.sqrt(mean_squared_error(y_test, pred))
    mae = mean_absolute_error(y_test, pred)
    mape = mean_absolute_percentage_error(y_test, pred)

    print(f"{ticker} → RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}")



===== Running LSTM + Technical Indicators for AAPL =====


/tmp/ipython-input-2010614393.py:76: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
AAPL → RMSE: 106.3394, MAE: 102.8886, MAPE: 0.5329

===== Running LSTM + Technical Indicators for MSFT =====


/tmp/ipython-input-2010614393.py:76: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step
MSFT → RMSE: 287.9121, MAE: 281.6845, MAPE: 0.7611

===== Running LSTM + Technical Indicators for GOOG =====


/tmp/ipython-input-2010614393.py:76: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
GOOG → RMSE: 72.8820, MAE: 67.7432, MAPE: 0.4531
